In [1]:
library(phyloseq)
library(microbiome)
library(tidyverse)
library(grid)
library(gridExtra)
library(vegan)
library(ranacapa)

Loading required package: ggplot2


microbiome R package (microbiome.github.com)
    


 Copyright (C) 2011-2022 Leo Lahti, 
    Sudarshan Shetty et al. <microbiome.github.io>



Attaching package: ‘microbiome’


The following object is masked from ‘package:ggplot2’:

    alpha


The following object is masked from ‘package:base’:

    transform


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ microbiome::alpha() masks ggplot2::alpha()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::lag()        masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    comb

In [2]:
metadata <- read.csv("../../inputs/raw_reads/metadata.csv", header = TRUE, row.names = 1)

In [3]:
bowtie_metaphlan <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Metaphlan4_Bowtie_report.csv", header = TRUE)
bowtie_kraken <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Kraken2_Bowtie_report.csv", header = TRUE)
bbmap_metaphlan <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Metaphlan4_BBmap_report.csv", header = TRUE)
bbmap_kraken <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Kraken2_BBmap_report.csv", header = TRUE)
bbmap_bracken <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Bracken_BBmap_report.csv", header = TRUE)
bowtie_bracken <-read.csv("../../results/Kraken_Bracken_Metaphlan_output/Bracken_Bowtie_report.csv", header = TRUE)

In [4]:
# removes clades with less than 50 reads

    bowtie_kraken <- bowtie_kraken %>%
        filter(reads_from_clade > 50)

    bbmap_kraken <- bbmap_kraken %>%
        filter(reads_from_clade > 50)

    bowtie_bracken <- bowtie_bracken %>%
        filter(reads_from_clade > 50)

    bbmap_bracken <- bbmap_bracken %>%
        filter(reads_from_clade > 50)

# Functions

In [5]:
fix_sample_names <-  function(report) {
    report <- report %>%
      mutate(sample = sub("^[0-9]+_", "", sample))    
}

In [6]:
metadata_preparation <- function(metadata, mapper, classifier){
    metadata <- metadata %>%
        rename(sample_name = colnames(metadata)) %>%
        mutate(mapper = mapper, classifier = classifier) %>%
        mutate(mapper_classifier = paste0(mapper, "_", classifier)) %>%
        sample_data()

    return(metadata)
}

In [7]:
create_phyloseq <- function(counts, metadata) {
    prepare_counts <- function(counts) {
    counts <- counts %>%
        select(-relative_abundance) %>%
        filter(domain == "Bacteria") %>%
        spread(key = sample, value = reads_from_clade, fill = 0) %>%
        mutate(rowname = paste0("otu", row_number())) %>%
        column_to_rownames("rowname")

    return(counts)

    }

    prepare_taxa <- function(counts){
    taxa <- counts %>%
        select(domain, phylum, class, order, family, genus, species) %>%
        mutate(across(everything(), ~na_if(., ""))) %>% 
        as.matrix() %>%
        tax_table()
    
    return(taxa)
    }

    prepare_otu <- function(counts) {
        otu <- counts %>%
            select(which(colnames(.) %in% rownames(metadata))) %>%
            as.matrix() %>%
            otu_table(taxa_are_rows = TRUE)

        return(otu)
    }

    counts <- prepare_counts(counts)
    otu <- prepare_otu(counts)
    taxa <- prepare_taxa(counts)

    phyloseq_object <- phyloseq(otu, taxa, metadata)

    return(phyloseq_object)
}


In [8]:
abundance_barplot <- function(phyloseq_object, clade, title) {
    # Aggregate data at the species level
    species_phyloseq <- tax_glom(phyloseq_object, clade)
    
    # Prune samples where the sum of counts at the species level is 0
    species_phyloseq <- prune_samples(sample_sums(species_phyloseq) > 0, species_phyloseq)
    
    # Create the plot using the original phyloseq_object but only with the samples remaining after pruning
    # This ensures that the plot reflects the original data, minus the pruned samples
    pruned_samples <- sample_names(species_phyloseq)
    pruned_phyloseq_object <- prune_samples(pruned_samples, phyloseq_object)
    
    plot <- plot_bar(pruned_phyloseq_object, fill = clade) + 
        theme_classic(base_size = 14) + # Increase the base text size
        theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 25), # Increase x-axis text size
              axis.title.x = element_text(size = 20), # Increase x-axis title size
              axis.title.y = element_text(size = 20),
              axis.text.y = element_text(size = 18),
              plot.title = element_text(size = 40, hjust = 0.5)) +
        labs(x = "", title = title)
    
    return(plot)
}

In [9]:
count_species_plot <- function(phyloseq_object, clade, title) {

    species_phyloseq <- tax_glom(phyloseq_object, clade)
    
    # Prune samples where the sum of counts at the species level is 0
    species_phyloseq <- prune_samples(sample_sums(species_phyloseq) > 0, species_phyloseq)

    df <- as.data.frame(otu_table(species_phyloseq))

    otu_df_filtered <- df %>%
        summarise_all(~sum(. != 0))

    # Convert the data to long format for plotting
    long_data <- tidyr::gather(otu_df_filtered, key = "Column", value = "NonZeroCount") 

    # Create the bar plot
    plot <- ggplot(long_data, aes(x = Column, y = NonZeroCount)) +
        geom_bar(fill = "#0d8ba4", color = "black", stat = "identity") +
        theme_classic(base_size = 14) + # Increase the base text size
        theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1, size = 12), # Increase x-axis text size
              axis.text.y = element_text(size = 12), # Increase y-axis text size
              axis.title.x = element_text(size = 14), # Increase x-axis title size
              axis.title.y = element_text(size = 14), # Increase y-axis title size
              plot.title = element_text(hjust = 0.5, size = 16), # Increase plot title size
              #legend.position = "none",
              panel.grid.major.y = element_line(color = "grey", size = 0.5)) +
        labs(x = "", y = "Number of Unique Species", title = title) +
        scale_y_continuous(limits = c(0, 18), breaks = seq(0, 18, by = 1))
    
    return(plot)
}

In [10]:
find_overlap <- function(phyloseq1, phyloseq2, clade) {

    # Convert phyloseq objects to dataframes
    df1 <- psmelt(phyloseq1)
    df2 <- psmelt(phyloseq2)
    
    # filter dataframes to keep only rows with non-zero abundance
    df1 <- df1 %>%
        filter(Abundance != 0)

    df2 <- df2 %>%
        filter(Abundance != 0)
    
    # Find the overlap between the two dataframes
    merged <- inner_join(df1, df2, by = c("Sample", clade))
    
    merged <- merged %>%
        select(Sample, clade) %>%
        distinct() %>%
        group_by(Sample) %>%
        summarise(uniques = n()) %>%
        as.data.frame()

    return(merged)

}

In [11]:
overlap_species_plot <- function(df, title) {
    # Create the bar plot
    plot <- ggplot(df, aes(x = Sample, y = uniques)) +
    geom_bar(fill = "#0d8ba4", color = "black", stat = "identity") +
    theme_classic() +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size=25), # X-axis labels
          axis.text.y = element_text(size=25), # Y-axis labels
          axis.title.x = element_text(size=25), # X-axis title
          axis.title.y = element_text(size=25), # Y-axis title
          plot.title = element_text(hjust=0.5, size=25), # Plot title
          #legend.position = "none",
          panel.grid.major.y = element_line(color="grey", size=0.5)) +
    labs(x="", y="Number of Unique Species", title="Unique Species per Sample") +
    scale_y_continuous(limits=c(0, 12), breaks=seq(0, 12, by=1))

    return(plot)
}

In [12]:
rarefication_plot <- function(phyloseq){

  otu_table <- vegan_otu(phyloseq)
  rarecurve <- rarecurve(otu_table, step=50,  cex=0.5, tidy = TRUE)

  plot <- ggplot(rarecurve, aes(x = Sample, y = Species, color = Site)) +
    geom_line(linewidth = 3) + # Increase line thickness
    theme_classic() + # Increase base text size for all text elements
    labs(x = "Sample", y = "Species", title = "Species by Sample Size across Samples", color = "Sample") +
    scale_x_log10(name = "Sample Size", limits = c(NA, 700000)) + 
    scale_y_continuous(name = "Species Count") + # Continuous y-axis
    theme(
      plot.title = element_text(size = 25), # Increase title text size
      axis.title = element_text(size = 25), # Increase axis labels text size
      axis.text = element_text(size = 20), # Increase axis tick text size
      legend.title = element_text(size = 20), # Increase legend title text size
      legend.text = element_text(size = 20),
      #legend.position = "none", # Increase legend text size
      panel.grid.major.y = element_line(color="grey", size=0.5),
      panel.grid.minor.y = element_line(color="grey", size=0.25)

    )
  return(plot)
}

# Data

In [13]:
rownames(metadata) <- sub("^[0-9]+_", "", rownames(metadata))

In [14]:
bowtie_metaphlan <- fix_sample_names(bowtie_metaphlan)
bowtie_kraken <- fix_sample_names(bowtie_kraken)
bbmap_metaphlan <- fix_sample_names(bbmap_metaphlan)
bbmap_kraken <- fix_sample_names(bbmap_kraken)
bbmap_bracken <- fix_sample_names(bbmap_bracken)
bowtie_bracken <- fix_sample_names(bowtie_bracken)

In [15]:
# prepare data for phyloseq creation
bowtie_metaphlan_metadata <- metadata_preparation(metadata, "bowtie2", "metaphlan4")
bowtie_kraken_metadata <- metadata_preparation(metadata, "bowtie2", "kraken2")
bowtie_bracken_metadata <- metadata_preparation(metadata, "bowtie2", "bracken")

bbmap_metaphlan_metadata <- metadata_preparation(metadata, "bbmap2", "metaphlan4")
bbmap_kraken_metadata <- metadata_preparation(metadata, "bbmap2", "kraken2")
bbmap_bracken_metadata <- metadata_preparation(metadata, "bbmap2", "bracken")

In [16]:
# create phyloseq file for each analysis
bowtie_metaphlan_phyloseq <- create_phyloseq(bowtie_metaphlan, bowtie_metaphlan_metadata)
bowtie_kraken_phyloseq <- create_phyloseq(bowtie_kraken, bowtie_kraken_metadata)
bowtie_bracken_phyloseq <- create_phyloseq(bowtie_bracken, bowtie_bracken_metadata)

bbmap_metaphlan_phyloseq <- create_phyloseq(bbmap_metaphlan, bbmap_metaphlan_metadata)
bbmap_kraken_phyloseq <- create_phyloseq(bbmap_kraken, bbmap_kraken_metadata)
bbmap_bracken_phyloseq <- create_phyloseq(bbmap_bracken, bbmap_bracken_metadata)


In [17]:
# create phyloseq files with relative abundance for all detected OTUs
bowtie_metaphlan_phyloseq_relative <- transform_sample_counts(bowtie_metaphlan_phyloseq, function(x) x / sum(x))
bowtie_kraken_phyloseq_relative <- transform_sample_counts(bowtie_kraken_phyloseq, function(x) x / sum(x))
bowtie_bracken_phyloseq_relative <- transform_sample_counts(bowtie_bracken_phyloseq, function(x) x / sum(x))

bbmap_metaphlan_phyloseq_relative <- transform_sample_counts(bbmap_metaphlan_phyloseq, function(x) x / sum(x))
bbmap_kraken_phyloseq_relative <- transform_sample_counts(bbmap_kraken_phyloseq, function(x) x / sum(x))
bbmap_bracken_phyloseq_relative <- transform_sample_counts(bbmap_bracken_phyloseq, function(x) x / sum(x))

In [18]:
# remove counts that did not identify specific specie
bowtie_metaphlan_phyloseq_species <- tax_glom(bowtie_metaphlan_phyloseq, taxrank = "species")
bowtie_kraken_phyloseq_species <- tax_glom(bowtie_kraken_phyloseq, taxrank = "species")
bowtie_bracken_phyloseq_species <- tax_glom(bowtie_bracken_phyloseq, taxrank = "species")

bbmap_metaphlan_phyloseq_species <- tax_glom(bbmap_metaphlan_phyloseq, taxrank = "species")
bbmap_kraken_phyloseq_species <- tax_glom(bbmap_kraken_phyloseq, taxrank = "species")
bbmap_bracken_phyloseq_species <- tax_glom(bbmap_bracken_phyloseq, taxrank = "species")

In [19]:
# create phyloseq files with relative abundance for OTUs at species level
bowtie_metaphlan_phyloseq_relative_species <- transform_sample_counts(bowtie_metaphlan_phyloseq_species, function(x) x / sum(x))
bowtie_kraken_phyloseq_relative_species <- transform_sample_counts(bowtie_kraken_phyloseq_species, function(x) x / sum(x))
bowtie_bracken_phyloseq_relative_species <- transform_sample_counts(bowtie_bracken_phyloseq_species, function(x) x / sum(x))

bbmap_metaphlan_phyloseq_relative_species <- transform_sample_counts(bbmap_metaphlan_phyloseq_species, function(x) x / sum(x))
bbmap_kraken_phyloseq_relative_species <- transform_sample_counts(bbmap_kraken_phyloseq_species, function(x) x / sum(x))
bbmap_bracken_phyloseq_relative_species <- transform_sample_counts(bbmap_bracken_phyloseq_species, function(x) x / sum(x))

# Plotting

In [20]:
options(repr.plot.width=25, repr.plot.height=25)

## Raw counts of all detected species

In [21]:
bowtie_metaphlan_raw_plot <- abundance_barplot(bowtie_metaphlan_phyloseq, "species", "Bowtie, Metaphlan4")
bowtie_kraken_raw_plot <- abundance_barplot(bowtie_kraken_phyloseq, "species", "Bowtie, Kraken2")
bowtie_bracken_raw_plot <- abundance_barplot(bowtie_bracken_phyloseq, "species", "Bowtie, Bracken")

bbmap_metaphlan_raw_plot <- abundance_barplot(bbmap_metaphlan_phyloseq, "species", "BBmap2, Metaphlan4")
bbmap_kraken_raw_plot <- abundance_barplot(bbmap_kraken_phyloseq, "species", "BBmap2, Kraken2")
bbmap_bracken_raw_plot <- abundance_barplot(bbmap_bracken_phyloseq, "species", "BBmap2, Bracken")

In [22]:
raw_abundance_plots <- arrangeGrob(bowtie_metaphlan_raw_plot,
                bowtie_kraken_raw_plot,
                bowtie_bracken_raw_plot,
                bbmap_metaphlan_raw_plot,
                bbmap_kraken_raw_plot,
                bbmap_bracken_raw_plot,
                ncol = 3)

## Relative abundance of all detected species

In [23]:
bowtie_metaphlan_relative_plot <- abundance_barplot(bowtie_metaphlan_phyloseq_relative, "species", "Bowtie, Metaphlan4")
bowtie_kraken_relative_plot <- abundance_barplot(bowtie_kraken_phyloseq_relative, "species", "Bowtie, Kraken2")
bowtie_bracken_relative_plot <- abundance_barplot(bowtie_bracken_phyloseq_relative, "species", "Bowtie, Bracken")

bbmap_metaphlan_relative_plot <- abundance_barplot(bbmap_metaphlan_phyloseq_relative, "species", "BBmap2, Metaphlan4")
bbmap_kraken_relative_plot <- abundance_barplot(bbmap_kraken_phyloseq_relative, "species", "BBmap2, Kraken2")
bbmap_bracken_relative_plot <- abundance_barplot(bbmap_bracken_phyloseq_relative, "species", "BBmap2, Bracken")

In [24]:
relative_abundance_plots <- arrangeGrob(bowtie_metaphlan_relative_plot,
                bowtie_kraken_relative_plot,
                bowtie_bracken_relative_plot,
                bbmap_metaphlan_relative_plot,
                bbmap_kraken_relative_plot,
                bbmap_bracken_relative_plot,
                ncol = 3)

## Raw counts of only identified species

In [25]:
bowtie_metaphlan_species_raw_plot <- abundance_barplot(bowtie_metaphlan_phyloseq_species, "species", "Bowtie, Metaphlan4")
bowtie_kraken_species_raw_plot <- abundance_barplot(bowtie_kraken_phyloseq_species, "species", "Bowtie, Kraken2")
bowtie_bracken_species_raw_plot <- abundance_barplot(bowtie_bracken_phyloseq_species, "species", "Bowtie, Bracken")

bbmap_metaphlan_species_raw_plot <- abundance_barplot(bbmap_metaphlan_phyloseq_species, "species", "BBmap2, Metaphlan4")
bbmap_kraken_species_raw_plot <- abundance_barplot(bbmap_kraken_phyloseq_species, "species", "BBmap2, Kraken2")
bbmap_bracken_species_raw_plot <- abundance_barplot(bbmap_bracken_phyloseq_species, "species", "BBmap2, Bracken")

In [26]:
raw_species_abundance_plots <- arrangeGrob(bowtie_metaphlan_species_raw_plot,
                bowtie_kraken_species_raw_plot,
                bowtie_bracken_species_raw_plot,
                bbmap_metaphlan_species_raw_plot,
                bbmap_kraken_species_raw_plot,
                bbmap_bracken_species_raw_plot,
                ncol = 3)

## Relative abundance of only identified species

In [27]:
bowtie_metaphlan_species_relative_plot <- abundance_barplot(bowtie_metaphlan_phyloseq_relative_species, "species", "Bowtie, Metaphlan4")
bowtie_kraken_species_relative_plot <- abundance_barplot(bowtie_kraken_phyloseq_relative_species, "species", "Bowtie, Kraken2")
bowtie_bracken_species_relative_plot <- abundance_barplot(bowtie_bracken_phyloseq_relative_species, "species", "Bowtie, Bracken")

bbmap_metaphlan_species_relative_plot <- abundance_barplot(bbmap_metaphlan_phyloseq_relative_species, "species", "BBmap2, Metaphlan4")
bbmap_kraken_species_relative_plot <- abundance_barplot(bbmap_kraken_phyloseq_relative_species, "species", "BBmap2, Kraken2")
bbmap_bracken_species_relative_plot <- abundance_barplot(bbmap_bracken_phyloseq_relative_species, "species", "BBmap2, Bracken")

In [28]:
relative_species_abundance_plots <- arrangeGrob(bowtie_metaphlan_species_relative_plot,
                bowtie_kraken_species_relative_plot,
                bowtie_bracken_species_relative_plot,
                bbmap_metaphlan_species_relative_plot,
                bbmap_kraken_species_relative_plot,
                bbmap_bracken_species_relative_plot,
                ncol = 3)

In [29]:
relative_species_abundance_plots_bowtie <- arrangeGrob(bowtie_metaphlan_species_relative_plot,
                bowtie_kraken_species_relative_plot,
                bowtie_bracken_species_relative_plot,
                ncol = 3)

In [30]:
ggsave(filename = "../../results/Phyloseq/Figures/relative_species_abundance_plots_bowtie.png", plot = relative_species_abundance_plots_bowtie,width = 45, height = 25, dpi = 300)


## Number of different species per sample

In [31]:
bowtie_metaphlan_species_counts_plot <- count_species_plot(bowtie_metaphlan_phyloseq_relative_species, "species", "Bowtie, Metaphlan4")
bowtie_kraken_species_counts_plot <- count_species_plot(bowtie_kraken_phyloseq_relative_species, "species", "Bowtie, Kraken2")
bowtie_bracken_species_counts_plot <- count_species_plot(bowtie_bracken_phyloseq_relative_species, "species", "Bowtie, Bracken")

bbmap_metaphlan_species_counts_plot <- count_species_plot(bbmap_metaphlan_phyloseq_relative_species, "species", "BBmap2, Metaphlan4")
bbmap_kraken_species_counts_plot <- count_species_plot(bbmap_kraken_phyloseq_relative_species, "species", "BBmap2, Kraken2")
bbmap_bracken_species_counts_plot <- count_species_plot(bbmap_bracken_phyloseq_relative_species, "species", "BBmap2, Bracken")

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


In [32]:
species_counts_plots <- arrangeGrob(bowtie_metaphlan_species_counts_plot,
                bowtie_kraken_species_counts_plot,
                bowtie_bracken_species_counts_plot,
                bbmap_metaphlan_species_counts_plot,
                bbmap_kraken_species_counts_plot,
                bbmap_bracken_species_counts_plot,
                ncol = 3)

In [33]:
bowtie <- find_overlap(bowtie_metaphlan_phyloseq_species, bowtie_bracken_phyloseq_species, "species")
bowtie_overlap_plot <- overlap_species_plot(bowtie, "Bowtie, overlap")

Warning message:
“Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(clade)

  # Now:
  data %>% select(all_of(clade))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.”


In [34]:
bbmap <- find_overlap(bbmap_metaphlan_phyloseq_species, bbmap_bracken_phyloseq_species, "species")
bbmap_overlap_plot <- overlap_species_plot(bbmap, "BBmap, overlap")

In [35]:
species_overlap_plots <- arrangeGrob(bowtie_overlap_plot,
                bbmap_overlap_plot,
                ncol = 2)

## Relative Genus abundance

In [36]:
# remove counts that did not identify specific specie
bowtie_metaphlan_phyloseq_genus <- tax_glom(bowtie_metaphlan_phyloseq, taxrank = "genus")
bowtie_kraken_phyloseq_genus <- tax_glom(bowtie_kraken_phyloseq, taxrank = "genus")
bowtie_bracken_phyloseq_genus <- tax_glom(bowtie_bracken_phyloseq, taxrank = "genus")

bbmap_metaphlan_phyloseq_genus <- tax_glom(bbmap_metaphlan_phyloseq, taxrank = "genus")
bbmap_kraken_phyloseq_genus <- tax_glom(bbmap_kraken_phyloseq, taxrank = "genus")
bbmap_bracken_phyloseq_genus <- tax_glom(bbmap_bracken_phyloseq, taxrank = "genus")

In [37]:
# create phyloseq files with relative abundance for OTUs at genus level
bowtie_metaphlan_phyloseq_relative_genus <- transform_sample_counts(bowtie_metaphlan_phyloseq_genus, function(x) x / sum(x))
bowtie_kraken_phyloseq_relative_genus <- transform_sample_counts(bowtie_kraken_phyloseq_genus, function(x) x / sum(x))
bowtie_bracken_phyloseq_relative_genus <- transform_sample_counts(bowtie_bracken_phyloseq_genus, function(x) x / sum(x))

bbmap_metaphlan_phyloseq_relative_genus <- transform_sample_counts(bbmap_metaphlan_phyloseq_genus, function(x) x / sum(x))
bbmap_kraken_phyloseq_relative_genus <- transform_sample_counts(bbmap_kraken_phyloseq_genus, function(x) x / sum(x))
bbmap_bracken_phyloseq_relative_genus <- transform_sample_counts(bbmap_bracken_phyloseq_genus, function(x) x / sum(x))

In [38]:
bowtie_metaphlan_genus_relative_plot <- abundance_barplot(bowtie_metaphlan_phyloseq_relative_genus, "genus", "Bowtie, Metaphlan4")
bowtie_kraken_genus_relative_plot <- abundance_barplot(bowtie_kraken_phyloseq_relative_genus, "genus", "Bowtie, Kraken2")
bowtie_bracken_genus_relative_plot <- abundance_barplot(bowtie_bracken_phyloseq_relative_genus, "genus", "Bowtie, Bracken")

bbmap_metaphlan_genus_relative_plot <- abundance_barplot(bbmap_metaphlan_phyloseq_relative_genus, "genus", "BBmap2, Metaphlan4")
bbmap_kraken_genus_relative_plot <- abundance_barplot(bbmap_kraken_phyloseq_relative_genus, "genus", "BBmap2, Kraken2")
bbmap_bracken_genus_relative_plot <- abundance_barplot(bbmap_bracken_phyloseq_relative_genus, "genus", "BBmap2, Bracken")

In [39]:
relative_genus_abundance_plots <- arrangeGrob(bowtie_metaphlan_genus_relative_plot,
                bowtie_kraken_genus_relative_plot,
                bowtie_bracken_genus_relative_plot,
                bbmap_metaphlan_genus_relative_plot,
                bbmap_kraken_genus_relative_plot,
                bbmap_bracken_genus_relative_plot,
                ncol = 3)

## Rarefication

In [40]:
bowtie_metaphlan_rarecurve <- rarefication_plot(bowtie_metaphlan_phyloseq_species)
bowtie_kraken_rarecurve <- rarefication_plot(bowtie_kraken_phyloseq_species)
bowtie_bracken_rarecurve <- rarefication_plot(bowtie_bracken_phyloseq_species)

bbmap_metaphlan_rarecurve <- rarefication_plot(bbmap_metaphlan_phyloseq_species)
bbmap_kraken_rarecurve <- rarefication_plot(bbmap_kraken_phyloseq_species)
bbmap_bracken_rarecurve <- rarefication_plot(bbmap_bracken_phyloseq_species)

Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 36”


Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 52”
empty rows removed

Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 56”
Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 32”
Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 52”
empty rows removed

Warning message in rarecurve(otu_table, step = 50, cex = 0.5, tidy = TRUE):
“most observed count data have counts 1, but smallest count is 56”


In [41]:
rarecurve_plots <- arrangeGrob(bowtie_metaphlan_rarecurve,
                bowtie_kraken_rarecurve,
                bowtie_bracken_rarecurve,
                bbmap_metaphlan_rarecurve,
                bbmap_kraken_rarecurve,
                bbmap_bracken_rarecurve,
                ncol = 3)

Warning message:
“Removed 10413 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 9576 rows containing missing values or values outside the scale range
(`geom_line()`).”


# Plot saving

In [42]:
# saving pcoa plots as png
ggsave(filename = "../../results/Phyloseq/Figures/raw_abundance_plots.png", plot = raw_abundance_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/relative_abundance_plots.png", plot = relative_abundance_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/raw_species_abundance_plots.png", plot = raw_species_abundance_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/relative_species_abundance_plots.png", plot = relative_species_abundance_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/species_counts_plots.png", plot = species_counts_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/species_overlap_plots.png", plot = species_overlap_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/relative_genus_abundance_plots.png", plot = relative_genus_abundance_plots,width = 45, height = 25, dpi = 300)
ggsave(filename = "../../results/Phyloseq/Figures/rarecurve_plots.png", plot = rarecurve_plots,width = 45, height = 25, dpi = 300)

In [43]:
# saving pcoa plots as pdf
ggsave(filename = "../../results/Phyloseq/Figures/raw_abundance_plots.pdf", plot = raw_abundance_plots, width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/relative_abundance_plots.pdf", plot = relative_abundance_plots, width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/raw_species_abundance_plots.pdf", plot = raw_species_abundance_plots, width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/relative_species_abundance_plots.pdf", plot = relative_species_abundance_plots, width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/species_counts_plots.pdf", plot = species_counts_plots, width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/species_overlap_plots.pdf", plot = species_overlap_plots,width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/relative_genus_abundance_plots.pdf", plot = relative_genus_abundance_plots,width = 45, height = 25)
ggsave(filename = "../../results/Phyloseq/Figures/rarecurve_plots.pdf", plot = rarecurve_plots,width = 45, height = 25)

In [44]:
saveRDS(bowtie_metaphlan_phyloseq, file = "../../results/Phyloseq/bowtie_metaphlan_phyloseq.rds")
saveRDS(bowtie_kraken_phyloseq, file = "../../results/Phyloseq/bowtie_kraken_phyloseq.rds")
saveRDS(bowtie_bracken_phyloseq, file = "../../results/Phyloseq/bowtie_bracken_phyloseq.rds")

saveRDS(bbmap_metaphlan_phyloseq, file = "../../results/Phyloseq/bbmap_metaphlan_phyloseq.rds")
saveRDS(bbmap_kraken_phyloseq, file = "../../results/Phyloseq/bbmap_kraken_phyloseq.rds")
saveRDS(bbmap_bracken_phyloseq, file = "../../results/Phyloseq/bbmap_bracken_phyloseq.rds")